In [1]:
%%capture

!pip install bs4
!pip install nbformat
!pip install pandas
!pip install geopandas
!pip install requests
!pip install --upgrade xlrd
!pip install openpyxl

In [2]:
%%capture

%run ./Tidy.ipynb

# Test d'un autre modèle: faire du clustering 

On veut ici utiliser la masse d'observation dont on dispose avec la prévalence pour toutes les drogues et toutes les classes d'âge.

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
from sklearn.cluster import KMeans  # pour kmeans
import seaborn as sns  # pour scatterplots

X_vars = ['Prevalence (%)','Drug']

# Une première idée rejetée

Nous avons en premier lieu essayé de prédire la légalisation ou non du cannabis au sein d'un même pays à l'aide d'un SVM prenant en entrée la prévalence dans chaque pays.

Cette première approche n'a pas porté les fruits escomptés et ce pour plusieurs raisons, la principale étant une inadéquation entre notre jeu de données (trop faible) et le modèle choisi. Cette dernière section nous permet d'illustrer les raisons de cet échec et de tirer des conclusions sur la démarche à adopter lors du choix et de la mise en place de modèles.

### Sélection des données

On s'était restreint dans le cadre de la prédiction de la légalisation ou non du cannabis dans chaque pays, au pourcentage d'individus ayant déjà consommé du cannabis dans leur vie pour chaque pays.
On se rend alors vite compte que cette restriction nous donne un jeu de données très faible (1 observation par pays, soit seulement 29 observations). Ce qui va poser problème par la suite.

In [4]:
import pandas as pd

# On récupère la table de légalisation que l'on a nettoyé en amont

legal = legal_pays_clean

# On prend la prévalence sur la vie entière

df = pd.read_csv('https://raw.githubusercontent.com/refouch/Projet_Python_2A/refs/heads/main/Data/EUDA/edr2024-gps-current-table-1.csv')

dfcurrent = df[df['Substance'] == 'Cannabis']

dfcurrentadults = dfcurrent[dfcurrent['Age'] == 'All adults (15-64)']

dfcurrentadultslf = dfcurrentadults[dfcurrentadults['Recall period'] == 'Lifetime']

# On effectue un merge les deux tables
dfadultslf_legal = dfcurrentadultslf.join(legal, on = 'Country')

In [5]:
# Maintenant on tente d'entraîner le modèle parce que psartek.

from sklearn import svm
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

# Recodage des variables catégorielles

dfadultslf_legal['Recreational_num'] = dfadultslf_legal['Recreational'].replace({'Decriminalized' : 2, 'Legal' : 1, 'Illegal' : 0})
dfadultslf_legal['Medical_num'] = dfadultslf_legal['Medical'].replace({'Decriminalized' : 2, 'Legal' : 1, 'Illegal' : 0})

X = dfadultslf_legal[['Prevalence (%)','Males (%)','Females (%)']]
Y = dfadultslf_legal['Recreational_num']

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2)

# Création du classifieur SVM avec le paramètre C=1
svm_classifier = svm.SVC(C=1, kernel='linear')

# Entraînement du modèle sur l'ensemble d'estimation
svm_classifier.fit(X_train, Y_train)

# Prédictions sur l'ensemble de test
Y_pred = svm_classifier.predict(X_test)

# Accuracy score = Normalization on correctly predicted outputs
accuracy_score = sklearn.metrics.accuracy_score(Y_test, Y_pred)
print(accuracy_score)

# Precision score = ratio tp / (tp + fp) -> ratio de vrais positifs contre faux positifs
precision_score = sklearn.metrics.precision_score(Y_test, Y_pred, average= 'micro')
print(precision_score)

#Recall 
recall_score = sklearn.metrics.recall_score(Y_test, Y_pred, average= 'micro')
print(recall_score)

#F1
f1_score = sklearn.metrics.f1_score(Y_test, Y_pred, average= 'micro')
print(f1_score)

0.5
0.5
0.5
0.5


/tmp/ipykernel_5649/185401596.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfadultslf_legal['Recreational_num'] = dfadultslf_legal['Recreational'].replace({'Decriminalized' : 2, 'Legal' : 1, 'Illegal' : 0})
/tmp/ipykernel_5649/185401596.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfadultslf_legal['Medical_num'] = dfadultslf_legal['Medical'].replace({'Decriminalized' : 2, 'Legal' : 1, 'Illegal' : 0})


# Encore des test: avec la table de lelay


In [10]:
import pandas as pd

# On récupère la table de légalisation que l'on a nettoyé en amont

legal = legal_pays_clean

# On prend la prévalence sur la vie entière

dfcurrent = test

# On effectue un merge les deux tables
dfadultslf_legal = test.join(legal, on = 'Country')

#On drop ensuite les pays pour lesquels on a pas d'observation sur les variables qui nous intéressent
dfadultslf_legal = dfadultslf_legal.dropna(subset=['Cannabis (global)','Cannabis (deja traite.es)','Cannabis (jamais traite.es)','Cannabis.jamais'])

dfadultslf_legal

,Country,Year of Treatment,All substances (global),Cannabis (global),All substances (deja traite.es),Cannabis (deja traite.es),All substances (jamais traite.es),Cannabis (jamais traite.es),Court / probation / police.global,General practitioner.global,...,Opioids.jamais,Cocaine.jamais,Stimulants.jamais,Hypnotics and Sedatives.jamais,Hallucinogens.jamais,Volatile Inhalants.jamais,Cannabis.jamais,Other substances.jamais,Recreational,Medical
0,Austria,2022.0,4709.0,1124.0,2756.0,458.0,1953.0,666.0,337.0,23.0,...,514.0,304.0,134.0,50.0,3.0,0.0,666.0,4.0,Decriminalized,Legal
1,Belgium,2022.0,12144.0,3501.0,7376.0,1524.0,4151.0,1850.0,NaN,NaN,...,216.0,1115.0,358.0,209.0,143.0,15.0,1850.0,29.0,Decriminalized,Legal
2,Bulgaria,2022.0,1879.0,109.0,1164.0,42.0,605.0,66.0,28.0,1.0,...,84.0,37.0,105.0,18.0,2.0,0.0,66.0,25.0,Illegal,Illegal
4,Cyprus,2022.0,884.0,340.0,308.0,66.0,535.0,251.0,139.0,0.0,...,83.0,125.0,74.0,1.0,NaN,1.0,251.0,NaN,Illegal,Legal
5,Czechia,2020.0,7283.0,1072.0,3336.0,469.0,2519.0,539.0,77.0,66.0,...,470.0,28.0,1317.0,129.0,14.0,3.0,539.0,19.0,Decriminalized,Legal
6,Denmark,2022.0,8241.0,3085.0,4408.0,1473.0,3368.0,1493.0,163.0,32.0,...,206.0,648.0,136.0,49.0,7.0,5.0,1493.0,65.0,Illegal,Legal
7,Estonia,2022.0,417.0,26.0,290.0,15.0,124.0,11.0,NaN,NaN,...,54.0,4.0,50.0,5.0,NaN,NaN,11.0,NaN,Decriminalized,Legal
8,Finland,2021.0,427.0,62.0,263.0,26.0,164.0,36.0,2.0,3.0,...,66.0,2.0,42.0,18.0,0.0,0.0,36.0,0.0,Illegal,Legal
9,France,2021.0,43078.0,19758.0,10628.0,3647.0,18195.0,10680.0,7325.0,651.0,...,2497.0,1911.0,494.0,472.0,183.0,15.0,10680.0,204.0,Illegal,Illegal
10,Germany,2022.0,40466.0,23228.0,16018.0,6921.0,22164.0,14981.0,5726.0,240.0,...,1601.0,2076.0,2825.0,545.0,74.0,5.0,14981.0,57.0,Legal,Legal


In [ ]:
# Maintenant on tente d'entraîner le modèle parce que psartek.

from sklearn import svm
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

# Recodage des variables catégorielles

dfadultslf_legal['Recreational_num'] = dfadultslf_legal['Recreational'].replace({'Decriminalized' : 1, 'Legal' : 1, 'Illegal' : 0})
dfadultslf_legal['Medical_num'] = dfadultslf_legal['Medical'].replace({'Decriminalized' : 1, 'Legal' : 1, 'Illegal' : 0})

X = dfadultslf_legal[['Cannabis (global)','Cannabis (deja traite.es)','Cannabis (jamais traite.es)','Cannabis.jamais']]
Y = dfadultslf_legal['Recreational_num']

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2)

# Création du classifieur SVM avec le paramètre C=1
svm_classifier = svm.SVC(C=1, kernel='linear')

# Entraînement du modèle sur l'ensemble d'estimation
svm_classifier.fit(X_train, Y_train)

# Prédictions sur l'ensemble de test
Y_pred = svm_classifier.predict(X_test)

# Accuracy score = Normalization on correctly predicted outputs
accuracy_score = sklearn.metrics.accuracy_score(Y_test, Y_pred)
print(accuracy_score)

# Precision score = ratio tp / (tp + fp) -> ratio de vrais positifs contre faux positifs
precision_score = sklearn.metrics.precision_score(Y_test, Y_pred, average= 'micro')
print(precision_score)

#Recall 
recall_score = sklearn.metrics.recall_score(Y_test, Y_pred, average= 'micro')
print(recall_score)

#F1
f1_score = sklearn.metrics.f1_score(Y_test, Y_pred, average= 'micro')
print(f1_score)

#psuh

/tmp/ipykernel_5649/4007374198.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfadultslf_legal['Recreational_num'] = dfadultslf_legal['Recreational'].replace({'Decriminalized' : 1, 'Legal' : 1, 'Illegal' : 0})
/tmp/ipykernel_5649/4007374198.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfadultslf_legal['Medical_num'] = dfadultslf_legal['Medical'].replace({'Decriminalized' : 1, 'Legal' : 1, 'Illegal' : 0})


0.6666666666666666
0.6666666666666666
0.6666666666666666
0.6666666666666666
